# Reinforcement Learning with Novelty seeking for modeling human behavior

In [4]:
import numpy as np
import random

# Параметры среды
GRID_SIZE = 5  # Размер сетки: GRID_SIZE x GRID_SIZE
GOAL = (4, 4)  # Цель, куда должен дойти агент
START = (0, 0)  # Стартовое положение агента

# Параметры RL
ALPHA = 0.1  # Скорость обучения
GAMMA = 0.9  # Коэффициент дисконтирования
EPSILON = 0.1  # Вероятность выбора случайного действия (Exploration rate)
NOVELTY_WEIGHT = 0.1  # Вес влияния компонента новизны

# Действия агента: вверх, вниз, влево, вправо
ACTIONS = ['up', 'down', 'left', 'right']

# Функция для выполнения действия в среде
def take_action(state, action):
    x, y = state
    if action == 'up' and x > 0:
        return (x-1, y)
    if action == 'down' and x < GRID_SIZE-1:
        return (x+1, y)
    if action == 'left' and y > 0:
        return (x, y-1)
    if action == 'right' and y < GRID_SIZE-1:
        return (x, y+1)
    return state  # Если действие невозможно, остаемся на месте

# Функция для получения награды
def get_reward(state):
    return 1 if state == GOAL else 0

# Инициализация Q-таблицы и таблицы посещаемости
Q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))
state_visits = np.zeros((GRID_SIZE, GRID_SIZE))  # Для учета "новизны"

# Выбор действия с учетом ε-жадной стратегии
def choose_action(state):
    if random.uniform(0, 1) < EPSILON:
        return random.choice(ACTIONS)  # Случайное действие
    else:
        x, y = state
        # Учет как стандартной Q-ценности, так и новизны
        novelty_bonus = NOVELTY_WEIGHT / (1 + state_visits[x, y])
        q_values_with_novelty = Q_table[x, y] + novelty_bonus
        return ACTIONS[np.argmax(q_values_with_novelty)]  # Жадный выбор действия

# Основной цикл обучения
EPISODES = 50
for episode in range(EPISODES):
    state = START
    total_reward = 0
    
    while state != GOAL:
        # Выбор действия
        action = choose_action(state)
        # Выполнение действия и переход в новое состояние
        new_state = take_action(state, action)
        # Получение награды
        reward = get_reward(new_state)
        total_reward += reward
        
        # Обновление таблицы посещений
        x, y = new_state
        state_visits[x, y] += 1  # Увеличиваем количество посещений состояния
        
        # Обновление Q-таблицы
        old_value = Q_table[state[0], state[1], ACTIONS.index(action)]
        next_max = np.max(Q_table[new_state[0], new_state[1]])
        Q_table[state[0], state[1], ACTIONS.index(action)] = \
            old_value + ALPHA * (reward + GAMMA * next_max - old_value)
        
        # Переход в новое состояние
        state = new_state

    print(f"Episode {episode+1}: Total Reward = {total_reward}")

# Демонстрация таблиц посетителей (учет новизны)
print("\n=== State Visits (новизна) ===")
print(state_visits)


Episode 1: Total Reward = 1
Episode 2: Total Reward = 1
Episode 3: Total Reward = 1
Episode 4: Total Reward = 1
Episode 5: Total Reward = 1
Episode 6: Total Reward = 1
Episode 7: Total Reward = 1
Episode 8: Total Reward = 1
Episode 9: Total Reward = 1
Episode 10: Total Reward = 1
Episode 11: Total Reward = 1
Episode 12: Total Reward = 1
Episode 13: Total Reward = 1
Episode 14: Total Reward = 1
Episode 15: Total Reward = 1
Episode 16: Total Reward = 1
Episode 17: Total Reward = 1
Episode 18: Total Reward = 1
Episode 19: Total Reward = 1
Episode 20: Total Reward = 1
Episode 21: Total Reward = 1
Episode 22: Total Reward = 1
Episode 23: Total Reward = 1
Episode 24: Total Reward = 1
Episode 25: Total Reward = 1
Episode 26: Total Reward = 1
Episode 27: Total Reward = 1
Episode 28: Total Reward = 1
Episode 29: Total Reward = 1
Episode 30: Total Reward = 1
Episode 31: Total Reward = 1
Episode 32: Total Reward = 1
Episode 33: Total Reward = 1
Episode 34: Total Reward = 1
Episode 35: Total Rewar

In [8]:
import gym
import numpy as np
import random
from collections import defaultdict

# Гиперпараметры
ALPHA = 0.1  # Скорость обучения
GAMMA = 0.99  # Коэффициент дисконтирования
EPSILON = 0.1  # Эпсилон для ε-жадной стратегии
NOVELTY_WEIGHT = 0.01  # Вес новизны
DISCRETE_BINS = (20, 20)  # Количество разбиений (дискретизация) для состояния
EPISODES = 5000  # Количество эпизодов для обучения

# Создаем среду
env = gym.make("MountainCar-v0")

# Функция дискретизации непрерывных состояний
def discretize_state(state, env_low, env_high, bins=DISCRETE_BINS):
    ratios = (state - env_low) / (env_high - env_low)
    discrete_state = (ratios * np.array(bins)).astype(int)
    return tuple(np.clip(discrete_state, 0, np.array(bins) - 1))

# Инициализация Q-таблицы
state_space_bins = DISCRETE_BINS
action_space = env.action_space.n
Q_table = np.zeros(state_space_bins + (action_space,))

# Таблица для учета количества посещений (новизна)
state_visits = defaultdict(int)

# Функция выбора действия с учетом ε-жадного подхода и компонента новизны
def choose_action(state):
    if random.uniform(0, 1) < EPSILON:
        return env.action_space.sample()  # Выбираем случайное действие
    else:
        novelty_bonus = np.array(
            [NOVELTY_WEIGHT / (1 + state_visits[state + (a,)]) for a in range(action_space)]
        )
        q_values_with_novelty = Q_table[state] + novelty_bonus
        return np.argmax(q_values_with_novelty)  # Выбираем действие с наилучшей скорректированной ценностью

# Основной цикл обучения
for episode in range(EPISODES):
    # Сбрасываем среду
    reset_output = env.reset()
    
    # Обработка нового формата Gym (означает, если env.reset() возвращает доп. значения, как словарь)
    if isinstance(reset_output, tuple): 
        state_raw, _ = reset_output
    else:  
        state_raw = reset_output

    state = discretize_state(state_raw, env.observation_space.low, env.observation_space.high)
    total_reward = 0
    done = False
    step = 0
    
    while not done:
        step += 1
        # Выбор действия
        action = choose_action(state)
        
        # Выполняем действие и переходим в новое состояние
        step_output = env.step(action)
        
        # Обработка нового формата Gym (возвращается кортеж: (next_state, reward, done, info))
        if isinstance(step_output, tuple):
            next_state_raw, reward, done, _ = step_output
        else:
            next_state_raw = step_output
        
        next_state = discretize_state(next_state_raw, env.observation_space.low, env.observation_space.high)
        
        # Награда модифицируется для поощрения поиска новизны
        if done and next_state_raw[0] >= env.goal_position:  # Если цель достигнута
            reward = 1
        else:
            reward -= 0.1  # Небольшое наказание за каждый шаг для улучшения обучения

        # Обновляем таблицу посещений
        state_visits[state + (action,)] += 1

        # Обновляем Q-значения
        old_value = Q_table[state + (action,)]
        next_max = np.max(Q_table[next_state])
        Q_table[state + (action,)] = old_value + ALPHA * (reward + GAMMA * next_max - old_value)

        # Переходим в новое состояние
        state = next_state
        total_reward += reward
            
        if done:
            break

    if episode % 100 == 0:
        print(f"Episode: {episode}, Reward: {total_reward}, Steps: {step}")

# Закрытие среды
env.close()

# Демонстрация количества посещений состояний
print("\n=== State-Action Visits ===")
for key, value in list(state_visits.items())[:10]:  # Показываем первые 10 записей
    print(f"State-Action: {key}, Visits: {value}")


AttributeError: module 'numpy' has no attribute 'bool8'